In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud


In [6]:
df = pd.read_json("MyData/StreamingHistory0.json")
df.tail()

,endTime,artistName,trackName,msPlayed
8836,2023-05-05 14:21,A$AP Ferg,"Dreams, Fairytales, Fantasies (feat. Brent Fai...",709
8837,2023-05-05 14:21,A$AP Ferg,Floor Seats,0
8838,2023-05-05 14:21,Kendrick Lamar,u,833
8839,2023-05-05 14:26,Baby Keem,trademark usa,270670
8840,2023-05-05 14:27,A$AP Ferg,Floor Seats,70224
